# 策略分析示例

调试策略可能很耗时。Freqtrade 提供了一些辅助函数用于可视化原始数据。
以下内容假设你使用 `SampleStrategy`，分析来自 Binance 的 5m 周期数据，并已将其下载到默认位置的数据目录中。
更多细节请参考 [官方文档](https://www.freqtrade.io/en/stable/data-download/)。

## 设置

### 将工作目录切换到仓库根目录

In [10]:
import os
from pathlib import Path


# 自动向上查找仓库根目录（Notebook 的工作目录可能不在仓库根目录）
# 只要找到下列任意一个标记文件/目录，就认为已到达项目根目录
MARKERS = ["pyproject.toml", "uv.lock", ".python-version", "AGENTS.md", ".git"]


def find_project_root(start: Path, max_depth: int = 10) -> Path:
    current = start.resolve()
    for _ in range(max_depth):
        if any((current / m).exists() for m in MARKERS):
            return current
        if current.parent == current:
            break
        current = current.parent
    raise FileNotFoundError(
        f"未能从 {start} 向上找到项目根目录（标记：{', '.join(MARKERS)}）"
    )


project_root = find_project_root(Path.cwd())
os.chdir(project_root)
print(project_root)


D:\Code\python\freqtrade_demo


### 配置 Freqtrade 环境

In [11]:
from pathlib import Path

from freqtrade.configuration.configuration import Configuration
from freqtrade.enums import RunMode


# Notebook 环境下建议显式指定 user_data_dir：
# 否则 Freqtrade 会默认使用 cwd/user_data（导致找不到目录或写到意外位置）
user_data_dir = Path.cwd()
config_file = user_data_dir / "config.json"

# 可选：使用其他配置文件
# config_file = user_data_dir / "configs/your_config.json"

config = Configuration(
    {"config": [str(config_file)], "user_data_dir": str(user_data_dir)},
    RunMode.OTHER,
).get_config()

# 定义一些常量
config["timeframe"] = "5m"
# 策略类名
config["strategy"] = "SampleStrategy"
# 数据所在目录
data_location = config["datadir"]
# 待分析交易对（此处只使用一个交易对）
pair = "BTC/USDT"


2026-01-05 10:44:14,890 - freqtrade.configuration.load_config - INFO - Using config: D:\Code\python\freqtrade_demo\config.json ...

2026-01-05 10:44:14,893 - freqtrade.loggers - INFO - Enabling colorized output.

2026-01-05 10:44:14,895 - freqtrade.loggers - INFO - Logfile configured

2026-01-05 10:44:14,896 - freqtrade.loggers - INFO - Verbosity set to 0

2026-01-05 10:44:14,899 - freqtrade.configuration.configuration - INFO - Using user-data directory: D:\Code\python\freqtrade_demo ...

2026-01-05 10:44:14,900 - freqtrade.configuration.configuration - INFO - Using data directory: D:\Code\python\freqtrade_demo\data\okx ...

2026-01-05 10:44:14,901 - freqtrade.exchange.check_exchange - INFO - Checking exchange...

2026-01-05 10:44:14,907 - freqtrade.exchange.check_exchange - INFO - Exchange "okx" is officially supported by the Freqtrade development team.

2026-01-05 10:44:14,908 - freqtrade.configuration.configuration - INFO - Using pairlist from configuration.

In [12]:
# 使用以上配置加载数据
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType


timeframe = config["timeframe"]

# Freqtrade 默认数据格式通常为 feather；若你下载时指定了其他格式，请在此处改成对应值
# 可选值：json / jsongz / feather / parquet
data_format = "feather"

candles = load_pair_history(
    datadir=data_location,
    timeframe=timeframe,
    pair=pair,
    data_format=data_format,
    candle_type=CandleType.SPOT,
)

# 确认加载成功
print(f"已从 {data_location} 加载 {pair} 的 {len(candles)} 行数据")
if candles.empty:
    raise RuntimeError(
        "未加载到任何K线数据，无法继续分析。\n"
        f"- datadir: {data_location}\n"
        f"- pair: {pair}\n"
        f"- timeframe: {timeframe}\n"
        f"- data_format: {data_format}\n"
        "请先下载数据，例如在仓库根目录运行：\n"
        f"uv run freqtrade download-data --userdir \".\" -c \"config.json\" --pairs \"{pair}\" -t \"{timeframe}\" --days 30"
    )

candles.head()


已从 D:\Code\python\freqtrade_demo\data\okx 加载 BTC/USDT 的 8668 行数据


,date,open,high,low,close,volume
0,2025-12-06 00:00:00+00:00,89330.9,89330.9,89180.0,89203.7,7.912459
1,2025-12-06 00:05:00+00:00,89203.5,89218.0,89180.0,89180.1,1.770009
2,2025-12-06 00:10:00+00:00,89180.0,89183.0,89040.3,89107.2,33.851069
3,2025-12-06 00:15:00+00:00,89107.1,89141.0,89070.7,89098.3,9.171595
4,2025-12-06 00:20:00+00:00,89098.1,89113.0,89043.0,89098.7,20.685164


## 加载并运行策略
* 每次修改策略文件后都需要重新运行

In [13]:
# 使用以上配置加载策略
from freqtrade.data.dataprovider import DataProvider
from freqtrade.resolvers import StrategyResolver


strategy = StrategyResolver.load_strategy(config)
strategy.dp = DataProvider(config, None, None)
strategy.ft_bot_start()

# 使用策略生成买卖信号
df = strategy.analyze_ticker(candles, {"pair": pair})
df.tail()

2026-01-05 10:44:14,944 - freqtrade.resolvers.iresolver - INFO - Using resolved strategy SampleStrategy from 'D:\Code\python\freqtrade_demo\strategies\sample_strategy.py'...

2026-01-05 10:44:14,945 - freqtrade.strategy.hyper - INFO - Loading parameters from file D:\Code\python\freqtrade_demo\strategies\sample_strategy.json

2026-01-05 10:44:14,947 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'timeframe' with value from the configuration: 5m.

2026-01-05 10:44:14,948 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'stake_currency' with value from the configuration: USDT.

2026-01-05 10:44:14,949 - freqtrade.resolvers.strategy_resolver - INFO - Override strategy 'stake_amount' with value from the configuration: unlimited.

...

2026-01-05 10:44:14,971 - freqtrade.strategy.hyper - INFO - Strategy Parameter: buy_rsi = 34

2026-01-05 10:44:14,971 - freqtrade.strategy.hyper - INFO - Strategy Parameter: exit_short_rsi = 30

2026-01-05 10:44:14,973 - freqtrade.strategy.hyper - INFO - Strategy Parameter: sell_rsi = 65

2026-01-05 10:44:14,974 - freqtrade.strategy.hyper - INFO - Strategy Parameter: short_rsi = 77

,date,open,high,low,close,volume,adx,rsi,fastd,fastk,...,sar,tema,htsine,htleadsine,enter_tag,enter_long,enter_short,exit_tag,exit_long,exit_short
8663,2026-01-05 01:55:00+00:00,93040.9,93050.9,92938.6,92963.9,12.350179,63.345754,68.609334,36.930589,7.405875,...,93403.050000,93100.860617,-0.051105,-0.742320,,NaN,NaN,,NaN,NaN
8664,2026-01-05 02:00:00+00:00,92964.0,93164.0,92964.0,93045.1,26.216720,63.696881,70.458049,25.975192,32.030075,...,93384.472000,93079.113058,-0.108925,-0.779921,,NaN,NaN,,NaN,NaN
8665,2026-01-05 02:05:00+00:00,93045.2,93110.0,92793.1,92859.0,35.815378,62.547181,61.516082,17.740854,13.786611,...,93366.637120,92971.043024,-0.172666,-0.818580,,NaN,NaN,,NaN,NaN
8666,2026-01-05 02:10:00+00:00,92858.1,92959.9,92837.0,92868.9,14.111092,61.479602,61.793868,21.135925,17.591088,...,93332.224893,92905.412424,-0.243606,-0.858060,,NaN,NaN,,NaN,NaN
8667,2026-01-05 02:15:00+00:00,92868.9,93000.0,92864.3,92992.9,15.715319,60.609296,65.183738,28.415556,53.868967,...,93299.877399,92924.754696,-0.316216,-0.894422,,NaN,NaN,,NaN,NaN


### 展示交易明细

* 注意：直接使用 `data.head()` 也可以，但多数指标在 dataframe 顶部会包含一些"启动期（startup）"数据。
* 一些可能的问题
    * dataframe 末尾存在 NaN 的列
    * 在 `crossed*()` 函数中参与比较的列单位/量纲完全不同
* 与完整回测的对比
    * `analyze_ticker()` 对单个交易对输出 200 个买入信号，并不意味着回测中就会产生 200 笔交易。
    * 例如你只用一个条件 `df['rsi'] < 30` 作为买入条件，它会为每个交易对连续产生多个"买入"信号（直到 RSI 回到 > 29）。机器人只会在这些信号中的第一个触发买入（并且需要仍有可用的交易名额，即 `max_open_trades`），或者在名额重新可用时，在中间某个信号处买入。  


In [14]:
# 输出结果
print(f"生成了 {df['enter_long'].sum()} 个入场信号")
data = df.set_index("date", drop=False)
data.tail()

生成了 91.0 个入场信号


,date,open,high,low,close,volume,adx,rsi,fastd,fastk,...,sar,tema,htsine,htleadsine,enter_tag,enter_long,enter_short,exit_tag,exit_long,exit_short
date,,,,,,,,,,,,,,,,,,,,,
2026-01-05 01:55:00+00:00,2026-01-05 01:55:00+00:00,93040.9,93050.9,92938.6,92963.9,12.350179,63.345754,68.609334,36.930589,7.405875,...,93403.050000,93100.860617,-0.051105,-0.742320,,NaN,NaN,,NaN,NaN
2026-01-05 02:00:00+00:00,2026-01-05 02:00:00+00:00,92964.0,93164.0,92964.0,93045.1,26.216720,63.696881,70.458049,25.975192,32.030075,...,93384.472000,93079.113058,-0.108925,-0.779921,,NaN,NaN,,NaN,NaN
2026-01-05 02:05:00+00:00,2026-01-05 02:05:00+00:00,93045.2,93110.0,92793.1,92859.0,35.815378,62.547181,61.516082,17.740854,13.786611,...,93366.637120,92971.043024,-0.172666,-0.818580,,NaN,NaN,,NaN,NaN
2026-01-05 02:10:00+00:00,2026-01-05 02:10:00+00:00,92858.1,92959.9,92837.0,92868.9,14.111092,61.479602,61.793868,21.135925,17.591088,...,93332.224893,92905.412424,-0.243606,-0.858060,,NaN,NaN,,NaN,NaN
2026-01-05 02:15:00+00:00,2026-01-05 02:15:00+00:00,92868.9,93000.0,92864.3,92992.9,15.715319,60.609296,65.183738,28.415556,53.868967,...,93299.877399,92924.754696,-0.316216,-0.894422,,NaN,NaN,,NaN,NaN


## 在 Jupyter Notebook 中加载已有对象

下面这些单元格假设你已经通过 CLI 生成了数据。  
它们可以帮助你更深入地查看结果，并进行分析；否则信息量过大时输出会很难阅读。

### 将回测结果加载为 pandas DataFrame

分析交易 DataFrame（下面的绘图也会用到）

In [15]:
from pathlib import Path

from freqtrade.constants import LAST_BT_RESULT_FN
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats


# backtest_dir 可以是目录，也可以直接指向某个 backtest-result 文件（.json / .zip）
# 如果 backtest_dir 指向目录，Freqtrade 会通过 backtest_results/.last_result.json 自动定位“最近一次回测结果”。
backtest_dir = Path(config["user_data_dir"]) / "backtest_results"

last_result_file = backtest_dir / LAST_BT_RESULT_FN
if backtest_dir.is_dir() and not last_result_file.is_file():
    raise RuntimeError(
        f"目录 {backtest_dir} 里还没有回测结果（缺少 {LAST_BT_RESULT_FN}）。\n"
        "请先在仓库根目录运行一次回测生成结果，例如：\n"
        f"uv run freqtrade backtesting --userdir \".\" -c \"config.json\" --strategy \"{config['strategy']}\""
    )

# 也可以手动指定某个结果文件，例如：
# backtest_dir = Path(config["user_data_dir"]) / "backtest_results/backtest-result-2026-01-05_10-00-00.zip"


In [16]:
# 你可以用以下命令获取完整的回测统计信息。
# 其中包含生成回测结果所需的全部信息。
stats = load_backtest_stats(backtest_dir)

strategy = config["strategy"]
# 所有统计信息都按策略分别提供，因此如果回测时使用了 `--strategy-list`，
# 这里也会体现出来。
# 示例用法：
print(stats["strategy"][strategy]["results_per_pair"])
# 获取本次回测使用的交易对列表
print(stats["strategy"][strategy]["pairlist"])
# 获取市场整体变化（回测期间所有交易对从开始到结束的平均变化）
print(stats["strategy"][strategy]["market_change"])
# 最大回撤（绝对值）
print(stats["strategy"][strategy]["max_drawdown_abs"])
# 最大回撤的起止时间
print(stats["strategy"][strategy]["drawdown_start"])
print(stats["strategy"][strategy]["drawdown_end"])


# 获取策略对比（仅在比较多个策略时有意义）
print(stats["strategy_comparison"])


2026-01-05 10:44:15,025 - freqtrade.data.btanalysis.bt_fileutils - INFO - Loading backtest result from D:\Code\python\freqtrade_demo\backtest_results\backtest-result-2026-01-05_10-35-01.zip

[{'key': 'BTC/USDT', 'trades': 6, 'profit_mean': 0.009985022153945656, 'profit_mean_pct': 1.0, 'profit_total_abs': 0.1996151, 'profit_total': 0.01996151, 'profit_total_pct': 2.0, 'duration_avg': '4 days, 14:50:00', 'wins': 6, 'draws': 0, 'losses': 0, 'winrate': 1.0, 'cagr': 0.28244063829873634, 'expectancy': 0.033269183333333334, 'expectancy_ratio': 100.0, 'sortino': -100, 'sharpe': 697.796150730556, 'calmar': -100, 'sqn': 394.7425, 'profit_factor': 0.0, 'max_drawdown_account': 0.0, 'max_drawdown_abs': 0.0}, {'key': 'TOTAL', 'trades': 6, 'profit_mean': 0.009985022153945656, 'profit_mean_pct': 1.0, 'profit_total_abs': 0.1996151, 'profit_total': 0.01996151, 'profit_total_pct': 2.0, 'duration_avg': '4 days, 14:50:00', 'wins': 6, 'draws': 0, 'losses': 0, 'winrate': 1.0, 'cagr': 0.28244063829873634, 'expectancy': 0.033269183333333334, 'expectancy_ratio': 100.0, 'sortino': -100, 'sharpe': 697.796150730556, 'calmar': -100, 'sqn': 394.7425, 'profit_factor': 0.0, 'max_drawdown_account': 0.0, 'm

In [17]:
# 将回测交易加载为 DataFrame
trades = load_backtest_data(backtest_dir)

# 按交易对统计退出原因的次数
trades.groupby("pair")["exit_reason"].value_counts()

2026-01-05 10:44:15,035 - freqtrade.data.btanalysis.bt_fileutils - INFO - Loading backtest result from D:\Code\python\freqtrade_demo\backtest_results\backtest-result-2026-01-05_10-35-01.zip

pair      exit_reason
BTC/USDT  roi            6
Name: count, dtype: int64

## 绘制每日收益 / 资金曲线

In [18]:
# 绘制资金曲线（第 1 天从 0 开始，逐日累加回测的每日收益）

import pandas as pd
import plotly.express as px

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_backtest_stats


# strategy = 'SampleStrategy'
# config = Configuration.from_files(["config.json"])
# backtest_dir = config["user_data_dir"] / "backtest_results"

stats = load_backtest_stats(backtest_dir)
strategy_stats = stats["strategy"][strategy]

df = pd.DataFrame(columns=["dates", "equity"], data=strategy_stats["daily_profit"])
df["equity_daily"] = df["equity"].cumsum()

fig = px.line(df, x="dates", y="equity_daily")
fig.show(renderer="browser")


2026-01-05 10:44:17,570 - freqtrade.data.btanalysis.bt_fileutils - INFO - Loading backtest result from D:\Code\python\freqtrade_demo\backtest_results\backtest-result-2026-01-05_10-35-01.zip

### 将实盘交易结果加载为 pandas DataFrame

如果你已经进行过实盘交易，并希望分析自己的表现

In [ ]:
from freqtrade.data.btanalysis import load_trades_from_db


# 从数据库读取交易记录
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# 展示结果
trades.groupby("pair")["exit_reason"].value_counts()

## 分析已加载交易的并行持仓情况
这在寻找最佳 `max_open_trades` 参数时很有用：可以先在回测中将 `max_open_trades` 设置得很高，然后结合该分析来评估实际需要的并行持仓数。

`analyze_trade_parallelism()` 会返回一个时间序列 DataFrame，其中包含 `open_trades` 列，用于表示每根 K 线对应的持仓数量。

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism


# 分析上述交易数据
parallel_trades = analyze_trade_parallelism(trades, "5m")

parallel_trades.plot()

## 绘制结果

Freqtrade 基于 plotly 提供交互式绘图能力。

In [20]:
from freqtrade.plot.plotting import generate_candlestick_graph


# 限制绘图区间，保持 plotly 绘制快速且响应灵敏

# 将交易记录筛选到单个交易对
trades_red = trades.loc[trades["pair"] == pair]

data_red = data["2019-06-01":"2019-06-10"]
# 生成 K 线图
graph = generate_candlestick_graph(
    pair=pair,
    data=data_red,
    trades=trades_red,
    indicators1=["sma20", "ema50", "ema55"],
    indicators2=["rsi", "macd", "macdsignal", "macdhist"],
)

2026-01-05 10:44:45,492 - freqtrade.plot.plotting - WARNING - No enter_long-signals found.

2026-01-05 10:44:45,494 - freqtrade.plot.plotting - WARNING - No exit_long-signals found.

2026-01-05 10:44:45,495 - freqtrade.plot.plotting - WARNING - No enter_short-signals found.

2026-01-05 10:44:45,496 - freqtrade.plot.plotting - WARNING - No exit_short-signals found.

2026-01-05 10:44:45,499 - freqtrade.plot.plotting - INFO - Indicator "sma20" ignored. Reason: This indicator is not found in your strategy.

2026-01-05 10:44:45,500 - freqtrade.plot.plotting - INFO - Indicator "ema50" ignored. Reason: This indicator is not found in your strategy.

2026-01-05 10:44:45,501 - freqtrade.plot.plotting - INFO - Indicator "ema55" ignored. Reason: This indicator is not found in your strategy.

In [21]:
# 在 Notebook 中内嵌显示图表
# graph.show()

# 在独立窗口中渲染图表
graph.show(renderer="browser")

## 绘制每笔交易平均收益的分布图

In [ ]:
import plotly.figure_factory as ff


hist_data = [trades.profit_ratio]
group_labels = ["profit_ratio"]  # 数据集名称

fig = ff.create_distplot(hist_data, group_labels, bin_size=0.01)
fig.show(renderer="browser")


如果你愿意分享更好的数据分析思路，欢迎提交 Issue 或 Pull Request 来完善本文档。